# `ad39_package` Documentation

Team39: Philipp Arens, Ben Ray, Andy Zhuo

## Abstract

This package provides a python implementation for automatic differentiation (AD).
The package supports both forward mode (FM) as well as reverse mode (RM) AD. To this end, the library supports differentiation 
for both univariate as well as multivariate inputs and supports both scalar and vector functions. That is, in the most general case
this library supports use cases of the form: $f: \mathbb{R}^m \mapsto \mathbb{R}^n$.

## Introduction

Derivatives play an essential role in many areas of science and engineering such as fluid dynamics, solving ODEs, optimization and - relatedly - machine learning. In general, there exist four main ways of computing derivatives in practice, that is: manual computation (i.e. computing and implementing derivatives by hand), symbolic differentiation (i.e. WolframAlpha or SymPy), numerical differentiation (i.e. finite difference method) and automatic differentiation (AD). Depending on the complexity of the target function, manual differentiation can require significant amounts of time and is often challenging to "debug". Symbolic differentiation alleviates this at the expense of potentially bloated/unitutitive expressions. If analytic derivative expressions are not needed, numerical differentiation, that is approximating derivatives through finite differences, could be considered. This, however, can lead to approximation/floating point errors if the step size is chosen inadequately. 

AD has emerged as a promising way of adressing many of these issues. Though not providing closed form/symbolic expressions, it allows us to compute derivatives with machine precision without introducing large computational overhead. Sparked by recent advances in machine learning, in particular deep neural networks, which use a subclass of AD (i.e. the backpropagation algorithm) in their training phase, AD has shown its potential across a variety of different aplications.

In this project we have developed a software package implementing thus far a subclass of AD, namely forward mode AD. We aim at making this software intuitive to use, following best practices in terms of relying on the python data model and style guides.

## Background

Central to automatic differentiation (AD) is use of the chain rule to decompose derivatives into elemtary binary operations (+, -, *, /, **) and unary operations (sin, cos, exp, log, etc.) Using the chain rule allows for two modes of evaluating a derivative:

1. Forward mode AD - the derivative is computed "bottom up" (i.e. fixing the independent variable(s), take the derivative of the innermost function first, and then move up taking derivatives of each sub-expression recursively)
2. Reverse mode AD - the derivative is computed "top down" (i.e. you fix the dependent variable, take the derivative of the outermost function first with respect to its sub-expressions, and then move inwards taking successive derivatives with respect to their inner sub-expressions).

AD can be generalized to both oridinary and partial derivatives using matrix products of Jacobians.

In forward mode AD, we commonly redefine our arithmetic to use dual numbers of the form $x + x'\epsilon$ where $\epsilon$ is a mathematical object not contained in the set of real numbers, with the property $\epsilon^2 = 0$. This allows us to store the decomposed derivative as a computational graph represented by dual numbers in each node. Despite this abstract implementation, we can still preserve the critical concept of "duck typing": as long as our dual numbers act like an input to their function (e.g. they can be added, multiplied, etc.) then the function will work. We similarly preserve the "duck typing" concept in our implementation of reverse mode AD, albeit using a node data structure, instead of dual numbers, under the hood (discussed below).

## How to use `ad39_package`

### Installation

We recommend installing the package into a virtual environment to avoid potential interferences with the user's system python environment. A virtual environment `test_env` can be created and activated as follows:

    python -m venv test_env
    source test_env/bin/activate

Users can install our package from the Test PyPi server using `pip`, whilst resolving dependencies from the PyPi server using the `--extra-index-url` flag:

    python -m pip install --extra-index-url https://pypi.org/simple/ -i https://test.pypi.org/simple/ ad39-package

Alternatively users can `git clone` directly from Team39's git repository using:

    git clone git@code.harvard.edu:CS107/team39.git

This assumes the dependencies (listed below) have already been installed.

### Dependencies
`ad39_package` requires the following packages:
* NumPy
* networkx
* matplotlib.pyplot

These dependencies will be resolved automatically if installation was completed with `pip` as described above.

### Demos

#### Import `ad39_package` and `numpy`

In [6]:
# Import forward and reverse mode classes (along with coressponding methods)
# from the two subpackages of `ad39_package`
from ad39_package.forward import FM
from ad39_package.reverse import RM

# Import numpy for later use
import numpy as np

#### Forward Mode

In [8]:
# Define target function: this can be a scalar- or vector-valued function,
# although vector-valued options must output a numpy array with the function
# operations defined within the array, i.e. 
# DO NOT USE `lambda x : np.array([1, 2]) + x`
# USE `lambda x : np.array([1 + x[0], 2 + x[1])`
f = lambda x: np.array([x[0] ** 2 + x[1] ** 2, FM.sqrt(x[0])])  

# Define example input vector
vec = np.array([4, 3])

# Instantiate an object for performing forward mode AD on the desired function
FMtest = FM(f)

# Evaluate the function by calling the `eval` method with argument `vec`
result = FMtest.eval(vec)
print(f'f([2, 3]) = {result}') # result = [25, 2]

# Compute the derivative of the function w.r.t. each of the two vector inputs
# evaluated at `vec` by calling the `grad` method with argument `vec` and
# `seed=[1,0]` or `seed=[0,1]` (if no seed is supplied, default will be to
# take derivative w.r.t. the first input - in this case x_0)
deriv_x0 = FMtest.grad(vec, seed=[1,0])
print(f'deriv w.r.t x_0 = {deriv_x0}') # deriv_x0 = [8, 0.25]

deriv_x1 = FMtest.grad(vec, seed=[0, 1])
print(f'deriv w.r.t x_1 = {deriv_x1}') # deriv_x1 = [6, 0]

# Compute the Jacobian evaluated at `vec` by calling the `jacobian`
# method with argument `vec`
jacob = FMtest.jacobian(vec)
print(f'The Jacobian of f([2, 3]) is {jacob}.') # jacob = [[8, 6], [0.25, 0]]

#### Extension 1: Forward Mode for multiple scalar and/or vector inputs
This was designed to help with usability or more complicated functions, where having only one input vector becomes unwieldy.

In [11]:
# Define target function with multiple vector and/or scalar inputs
f = lambda x, y: x[0] + y ** 2

# Define example inputs
vec = np.array([1, 2])
scalar = 3

# Instantiate an object for performing forward mode AD on the desired function
FMtest = FM(f)

# Evaluate the function with multiple inputs passed as a tuple
result = FMtest.eval((vec, scalar))
print(f'f([1, 2], 3) = {result}') # result = 10

# Compute the derivative of the function w.r.t. each of the two vector inputs
# and one scalar input evaluated at the tuple of inputs
deriv_x0 = FMtest.grad((vec, scalar), seed=[1,0,0])
print(f'deriv w.r.t x_0 = {deriv_x0}') # deriv_x0 = 1

deriv_x1 = FMtest.grad((vec, scalar), seed=[0, 1, 0])
print(f'deriv w.r.t x_1 = {deriv_x1}') # deriv_x1 = 0

deriv_y = FMtest.grad((vec, scalar), seed=[0,0,1])
print(f'deriv w.r.t y = {deriv_y}') # deriv_y = 6

# Compute the Jacobian evaluated at the tuple of inputs
jacob = FMtest.jacobian((vec, scalar))
print(f'The Jacobian of f([1, 2], 3) is {jacob}') # jacob = [1, 0, 6]


#### Extension 2: Reverse Mode

In [15]:
# Define target function: this can be a scalar- or vector-valued function,
# although vector-valued options must output a numpy array with the function
# operations defined within the array, i.e. 
# DO NOT USE `lambda x : np.array([1, 2]) + x`
# USE `lambda x : np.array([1 + x[0], 2 + x[1])`
f = lambda x: np.array([x[0] ** 2 + x[1] ** 2, RM.sqrt(x[0])])  

# Define example input vector
vec = np.array([4, 3])

# Instantiate an object for performing forward mode AD on the desired function
RMtest = RM(f)

# Evaluate the function by calling the `eval` method with argument `vec`
result = RMtest.eval(vec)
print(f'f([2, 3]) = {result}') # result = [25, 2]

# Compute the derivative of the function w.r.t. each of the two vector inputs
# evaluated at `vec` by calling the `grad` method with argument `vec`. Note,
# that no seed vector is required; `RM.grad` returns the entire gradient w.r.t.
# each input.
gradient_vec = RMtest.grad(vec)
print(f'gradient = {gradient_vec}') # gradient = [[8, 0.25], [6, 0]]

# Compute the Jacobian evaluated at `vec` by calling the `jacobian`
# method with argument `vec`
jacob = FMtest.jacobian(vec)
print(f'The Jacobian of f([2, 3]) is {jacob}.') # jacob = [[8, 6], [0.25, 0]]

#### Extension 3: Computational Graph for Reverse Mode (Beta Version)

In [ ]:
# Draw the computational graph of the function evaluated at the vector defined above
RMtest.graph(vec)

# Input nodes are displayed in red, output nodes are displayed in green

## Software Organization

### Structure

    team39
        ├── LICENSE
        ├── README.md
        ├── dist
        ├── docs
        │   ├── documentation.ipynb
        │   ├── milestone1.ipynb
        │   ├── m2-install.sh
        │   ├── milestone2.ipynb
        │   └── milestone2_progress.md
        ├── pyproject.toml
        ├── setup.cfg
        ├── src
        │   └── ad39_package
        │       ├── __init__.py
        │       ├── core.py
        │       ├── forward
        │       │   ├── __init__.py
        │       │   ├── forward_core.py
        │       │   └── forward_helpers.py
        │       └── reverse
        │           ├── __init__.py
        │           ├── reverse_core.py
        │           └── reverse_helpers.py
        └── tests
            ├── check_coverage.py
            ├── check_coverage.sh
            ├── htmlcov
            ├── run_tests.sh
            ├── test_core.py
            ├── test_forward_core.py
            ├── test_forward_helpers.py
            ├── test_reverse_core.py
            └── test_reverse_helpers.py

### Modules

On the root level our project contains basic package files like the `LICENCE`,  `README`, a `docs` directory containing a detailed project documentation and as well as the build configuration files `pyproject.toml` and `setup.cfg`.

The actual implementation files can be found in the `src` directory, which contains the main `ad39_package`. In the module `core.py` we implements the base `AD` class, which our forward mode and reverse mode classes inherit from. 

We further split our software into a `forward` and `reverse` subpackage, each of which contain a `*_core.py` module providing the main user interface classes, as well as `*_helper.py` modules implementing the dual numbers class used in `FM` as well as the node class used in `RM`. These helper classes also take care of the operator overloading for elementary and unary operators.

### Tests

Our test suite is located in a designated `tests` directory allowing straightforward unit and integration testing using pytest.

To test our packages functionality using pytest we can execute a test driver script as follows: `./run_tests pytest`. This script runs all the test scripts (`test_*.py`) written in our test directory and is mainly used in our CI pipeline whenever we push to the `master` branch.

Similarly, we check our tests' coverage by executing `./run_tests.sh CI --cov=ad39_package --cov-report=html:htmlcov`. This generates a report on the coverage in `html` format, which updates every time we push to the master branch. The coverage report can be found at this link: https://code.harvard.edu/pages/CS107/team39/.

Alternatively if running tests locally the user can simply execute the command

    ./check_coverage.sh pytest

Running this generates the following report:

TOTAL

    Name                                          Stmts   Miss  Cover   Missing
    ---------------------------------------------------------------------------
    src/ad39_package/__init__.py                      4      0   100%
    src/ad39_package/core.py                         50      2    96%   60, 108
    src/ad39_package/forward/__init__.py              2      0   100%
    src/ad39_package/forward/forward_core.py        207      5    98%   119-120, 130-131, 521
    src/ad39_package/forward/forward_helpers.py      62      5    92%   19, 27, 37-40
    src/ad39_package/reverse/__init__.py              2      0   100%
    src/ad39_package/reverse/reverse_core.py        249     46    82%   28, 136, 193-243, 569
    src/ad39_package/reverse/reverse_helpers.py      94      4    96%   23, 30, 33-35
    ----------------------------------------------------------------------------
    TOTAL                                           670     62    91%

Note that the relatively low coverage in `reverse_core.py` is due to the currenlty experimental graph feature which will be extended in future work.

### Package Distribution & How To Contribute

If you wish to contribute to this package we recommend creating a fork from https://code.harvard.edu/CS107/team39.git and submitting a pull request with a clear discription of the problem/desired feature you think is missing along with your proposed solution.

## Implementation

### Core Data Structures

`FM` or `RM` class instances are created for specific functions, which the user may be interested in evaluating or differentiating using forward or reverse mode respectively. Hence, the `FM` and `RM` class has just one public attribute for these specific functions `self.f`. Thus, the `FM` and `RM` classes may be instantiated on any kind of function object, i.e. an anonymous/lambda function or non-anonymous function in Python. The `AD` base class and its methods can be found in `core.py`, and the `FM` and `RM` classes which inherit from it can be found in `forward_core.py` and `reverse_core.py` respectively.

The `DualNumbers` class has two key attributes `self.real` and `self.dual`. Intuitively, these describe the real and dual parts of a dual number respectively. The class also implements operator overloading for standard binary, reverse binary, and unary operators. The `DualNumber` class an its methods can be found in `forward_helpers.py`.

The reverse mode implementation of AD relies on a directed computational graph represented as a set of instantiated `Node` objects. Each instance of the `Node` class stores, upon initialization: 
1. `val` - the function's value evaluated at this step in the computation
2. `children` - an array of children nodes, used to accumulate the current node's adjoint
3. `partials` - an array of the node's local partial derivatives with respect to the parent(s) of the node
4. `adjoints` - the function's partial derivative with respect to the current node, initialized as 0 until a reverse pass is made.
5. `op` - the operation used to obtain this function's value at this step in the computation (which is used for our graphing extension only)

The use of a digraph data structure for reverse mode allows us to traverse through each intermediate operation in a single forward pass to evaluate the function and compute its partials. Performing a single reverse pass by recursively traversing through each node and its children and using the local partials to build up the adjoints allows us to compute the partial for each input variable and thus thereby the function's gradient. In order to perform a forward pass on the digraph, we built off the operator overloads for `DualNumber` to the `Node` class. While the `val` and `partials` attributes are computed similarly to `real` and `dual` in the current `DualNumber` class, we extended the operator overloads for `Node` to account for the next intermediate operations, represented as the children of a node. 

### Classes and Attributes

In [ ]:
"""
Full implementation of base `AD` class in `core.py`. `FM` and `RM` class inherit from this base class, 
and can be found in `forward_core.py` and `reverse_core.py` respectively.
"""

def __init__(self, f):
    """Initialize an instance of an automatic differentiation object given a scalar-valued function `f`.

    Parameters
    ----------
    f : (int, float, np.ndarry, tuple) -> (int, float, np.ndarry)
        Function to which methods from the AD class are applied.
        Note the `reverse_mode` method excludes functionality for 
        functions taking tuple as input.
    
    See Also
    ----------
    ad39_package.forward.FM
    ad39_package.reverse.RM
    """
    self.f = f

    # Store number of args the function expects
    self._nargs = f.__code__.co_argcount


def eval(self, inputs):
    """Evaluate the value of `self.f` at `inputs`."""
    raise NotImplementedError

def grad(self, inputs, *args):
    """Compute the derivative of `self.f` evaluated at `inputs` using AD."""
    raise NotImplementedError

def jacobian(self, inputs):
    """Compute the Jacobian of `self.f` evaluated at `inputs` using AD."""
    raise NotImplementedError

# More complex functions like `sin` / `exp` / etc. are implemented as standalone 
# static methods accessible to the user like the following `sqrt` method
@staticmethod
def sqrt(x):
    """Compute sqrt of `x` in a function. Output is not accessible unless `AD.eval` method is called."""
    raise NotImplementedError

As the chain rule holds for dual numbers we know that for $f(z) = f(a + b \epsilon) = f(a) + f'(a)b \epsilon$. Thus, abiding to the principles of duck typing, we overload the operators for the `DualNumber` class:

In [ ]:
"""
Full implementation in `forward_helpers.py`. Note, that none of these functions are required to be used 
directly by a user of `ad39_package`.
"""

class DualNumber:
    _supported_scalars = (float, int)

    def __init__(self, real, dual=1.0):
        self.real = real
        self.dual = dual

    def __repr__(self):
        pass

    # Binary operations
    def __add__(self, other):
        pass

    def __sub__(self, other):
        pass

    def __mul__(self, other):
        pass

    def __truediv__(self, other):
        pass
    
    def __pow__(self, other):
        pass

    # Reverse binary operations
    def __radd__(self, other):
        pass

    def __rmul__(self, other):
        pass

    def __rsub__(self, other):
        pass

    def __rtruediv__(self, other):
        pass
    
    # Unary operations
    def __neg__(self):
        pass

Similarly, we preserve the concept of duck typing with our operator overloading for the `Node` class:

In [ ]:
"""
Full implementation in `reverse_helpers.py`. Note, that none of these functions are required to be used 
directly by a user of `ad39_package`.
"""

class Node:
    _supported_scalars = (float, int)

    def __init__(self, val, partials = 1, children = None, adjoints = 0, op = None):

        self.val = val
        self.partials = [partials] if isinstance(partials, self._supported_scalars) else partials
        self.adjoints = adjoints
        self.children = children if children is not None else []
        self.op = op

    # Same operator overloading as in `DualNumber` class

### External Libraries

* NumPy
* networkx
* matplotlib.pyplot

## Extension

This is what we stated in our Milestone 2 document with regards to our extension:

"_Our current implementation of Automatic Differentiation uses Dual Numbers to compute the derivative of a scalar function $f: \mathbb{R} \rightarrow \mathbb{R}$. While this suffices for most use cases, we will extend our package to support multivariate vector functions $f: \mathbb{R}^m \rightarrow \mathbb{R}^n$. However, as the input dimension grows increasingly large, using the forward mode variant of AD becomes inefficient as the number of passes grows proportionate to the number of inputs. Accordingly, in order to acccommodate multiple inputs while accounting for efficiency, we will also implement reverse mode for multivariate functions, which bounds the number of forward passes to one but now requires the storage of all intermediate operations as variable-nodes in a computational graph._

_Finally, if time allows, with a computational graph constructed via the implemented `Node` class, we aim to provide functionality for drawing the computational graph using the `NetworkX` Python package._"

### Extension 1: Forward Mode for multiple scalar and/or vector inputs
The package already supported functions of the form `lambda x : f(x)` where `x` was a scalar or vector input, but we thought it may become unwieldy to append many inputs to one `x` vector. Thus, we added support for functions of the form `lambda x, y, z : x[0] + y - z`. This is a much neater implementation that making:
`v = [x[0], y, z]` and writing a function `lambda v : v[0] + v[1] - v[2]`, or at least is much neater for more complicated functions!

### Extension 2: Reverse Mode
As previously stated, as the input dimension of a function grows increasingly large, using the forward mode variant of AD becomes inefficient as the number of passes grows proportionate to the number of inputs. Accordingly, we implemented reverse mode for multivariate functions, which bounds the number of forward passes to one but stores all intermediate operations as variable-nodes in a computational graph.

### Extension 3: Computational Graph for Reverse Mode (Beta Version)
We provided a beta version of our work in progress graphing feature for reverse mode, which we said we would start if time allowed. The method `graph` in the `RM` class can be found in `reverse_core.py` So far, the graph is plotted with red nodes representing the inputs and green nodes the outputs, but would understandably become messy for more complicated functions. An example of our current implementation is provided below - as can be seen the labelling of the nodes based on hierarchical presedence is not yet fully implemented. This graph represents the function:
```python 
f = lambda x: x[0] ** 2 + 2*x[1]
```

<div><img src="./graph.png" width="700"/></div>

## Borader Impact and Inclusivity Statement

#### Broader Impact

Automatic Differentiation (AD) has emerged as an indispendable tool in nearly all computational fields today. As such,
it has far reaching implications on many areas of our daily lives, with many people not being aware of this. The arguably most 
prominent example of this is AD's role in modern machine learning algorithms, first and foremost it's application in training
deep neural networks. That is, training neural networks utilizes a form of reverse mode AD called the backpropagation algorithm. 
Consequently, AD's is indirectly impacting many fields in modern society in which deep learning is being utilized such as face recognition software,
loan-approval or other rating algorithms. Predictions from such algorithms are known to fail in certain cases and can produce biased results.
It is therefore critical to always question the results obtained by such algorithms and not to base decisions solely on what the results suggest.

#### Inclusivity

We explicitly welcome individuals from any background, ethnicity or sexual orientation to contribute to this project. We realize it is often easier said that done. If you are interested in joining our core developer team or learn about other ways to contribute please don't hesiatate to reach out to us. As our code base and documentation is currently written entire;y in English we would particularly welcome contributions in helping translate our documentation into other languages to make it easier to use for a broader audience. 

## Future Work

There are two core avenues for improving the functionality of the existing functions supplied by the `ad39_package` and numerous options for improvement beyond what is already available:

1. We would hope to extend the functionality of Extension 1 to reverse mode, allowing the `RM` class to be instantiated for functions with multiple scalar and/or vector inputs.

2. We plan to improve our beta mode for visualizing the computational graph for AD, first for reverse mode (Extension 3) and then for forward mode. To this end we aim to provide further functionality for drawing the computational graph using the `NetworkX` Python package.

Finally, we actively welcome contributions to this package by forking from https://code.harvard.edu/CS107/team39.git and submitting a pull request with proposed additions. We have alluded to some of the possible extensions in this documentation with optimization algorithms, neural networks, and machine learning all potentially standing to benefit from AD.

## Appendix

### License

We decided to go with an MIT (i.e. copyright) license as it permits reuse within proprietary software, provided that all copies of the software or its substantial portions include a copy of the terms of the MIT License and also our original copyright notice. Since we do not hope to commercialize this project, we are happy for others to use it. However, we feel that including the original copyright notice in all derived projects is important, so that future developers understand that the `ad39` package was designed as an educational project (and not more than that), i.e. we want to set developers' expectations for using the `ad39` package by telling them the provenance of the package.

### Milestone 1 Feedback (some of which no longer applies)
- Background
    - We explained the difference between forward and reverse node as computing the derivative "bottom up" and "top down" rather than "from the inside out" and "from the outside in", given the intuition that expressions can be thought of as syntax trees.
    - We clarified that $\epsilon$ is a mathematical object not contained in the set of real numbers, to clarify what we had previously described as an "abstract number".

- How To Use
    - Summarized usage in a code snippet.
    - Changed how you initialized variables as nodes, removing the need for providing a name to the node (which could conflict across variables): i.e. before you initialized as follows `x1 = Node("x1")`, now it is just `x1 = Node()`.

- Software Organization
    - Added a note that we may need more modules in `ad39_package` as the project grows.

- Implementation
    - Removed `node.child` from our implementation of the `Node` class following feedback that "as long as each node keeps reference of its parent(s), you can access the whole graph with the output nodes as a handle." Also removed `node.name` as per earlier feedback - this is no longer necessary, and if we ever need to give a unique ID to nodes, it will be done entirely on the backend.
    - Removed the unintuitive `ForwardMode` class, and instead defined an `AD` class, which now includes: (1) a new `forward_mode` method, (2) the `grad` method (moved from the `Node` class to here, but no longer a static method), and (3) any other methods that will be included in our extension, e.g. `draw` for drawing the computational graph. Note, that the `AD` class object is initialized with the function on which AD will be performed, and we may use a further `Tree` class (or just a list of `Node` objects) under the hood to store the nodes created during the computation.